In [ ]:
# GEREKLI KUTUPHANELERIN EKLENMESI

import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import torch

In [ ]:
# CSV DOSYASININ OKUNMASI

dframe = pd.read_csv("/content/vodafone_data.csv", encoding='utf_8', on_bad_lines = 'skip', sep = ';')
dframe

,Explanation,Target
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0
2,Vodafone benim için bitti artık her yerde çeki...,0
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0
...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0


In [ ]:
# TITLE VE LINK SUTUNLARININ TABLODAN CIKARILMASI

#dframe = dframe.iloc[300:]
#dframe = dframe.drop(columns="Title",axis = 1)
#dframe = dframe.drop(columns="Link",axis=1)
#dframe = dframe.sample(frac = 1)
dframe

,Explanation,Target
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0
2,Vodafone benim için bitti artık her yerde çeki...,0
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0
...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0


In [ ]:
# TABLODAKI DUYGULARIN SAYISI

print("0: Olumsuz", "1: Olumlu", "2: Nötr\n")
dframe["Target"].value_counts()

0: Olumsuz 1: Olumlu 2: Nötr



,count
Target,
0,2906
1,1203
2,723


In [ ]:
# METINDEN DOLGU SOZCUKLERININ CIKARILMASI


In [ ]:
# METINDEN NOKTALAMA ISARETLERINI, RAKAMLARI CIKARMA

def clean_text(text):
    text = " ".join(str(text).split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return "".join(text)

dframe["Clean"] = dframe.apply(lambda row: clean_text(row["Explanation"]), axis=1)
dframe

,Explanation,Target,Clean
0,Vodafone'a ıyy diyen hayatında hiç vodafone ku...,1,vodafonea ıyy diyen hayatında hiç vodafone kul...
1,Her yerde çekiyor diye geçtiğim güne lanet ols...,0,her yerde çekiyor diye geçtiğim güne lanet ols...
2,Vodafone benim için bitti artık her yerde çeki...,0,vodafone benim için bitti artık her yerde çeki...
3,Vodafone Sizin de içine mi sızdılar yoksa bili...,0,vodafone sizin de içine mi sızdılar yoksa bili...
4,Saf olmamak gerekir. Güvenlik açığı demek olay...,0,saf olmamak gerekir güvenlik açığı demek olayı...
...,...,...,...
4827,Tam iki ay oldu hatalı faturam düzeltilmedi. İ...,0,tam iki ay oldu hatalı faturam düzeltilmedi i̇...
4828,Müşteri hizmeti olarak arıyorsunuz 50 gb hafız...,0,müşteri hizmeti olarak arıyorsunuz gb hafıza ...
4829,Izmirin ciglisinde çarşının ortasında şebekeni...,0,izmirin ciglisinde çarşının ortasında şebekeni...
4830,TV’lerle uğraşmaktan bıktığım için böyle zaten...,0,tv’lerle uğraşmaktan bıktığım için böyle zaten...


In [ ]:
# SATIRLARI NUMPY DIZISI YAPMA

X = dframe["Clean"]
y = dframe["Target"]

In [ ]:
# TEST VE EGITIM VERILERININ AYRILMASI

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

In [ ]:
# BERT TOKENIZER EKLEME

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [ ]:
# BERT TOKENIZER UYGULAMA

max_len= 1024

X_train_encoded = tokenizer(list(X_train), truncation=True, padding=True, max_length = max_len)

X_test_encoded = tokenizer(list(X_test), truncation=True, padding=True, max_length = max_len)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoded),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test_encoded),
    y_test
))

# Shuffle and batch the data
train_dataset = train_dataset.shuffle(len(X_test)).batch(16)
val_dataset = val_dataset.batch(16)


In [ ]:
# SINIFLANDIRMA MODELINI YUKLEME

model = TFBertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels = 3)

tf_model.h5:   0%|          | 0.00/545M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# MODELI COMPILE ETME

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# MODELI EGITME

history = model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
212/212 [==============================] - 393s 2s/step - loss: 0.6279 - accuracy: 0.7389 - val_loss: 0.5354 - val_accuracy: 0.7766
Epoch 2/10
212/212 [==============================] - 351s 2s/step - loss: 0.3334 - accuracy: 0.8788 - val_loss: 0.3871 - val_accuracy: 0.8552
Epoch 3/10
212/212 [==============================] - 350s 2s/step - loss: 0.1920 - accuracy: 0.9347 - val_loss: 0.4316 - val_accuracy: 0.8566
Epoch 4/10
212/212 [==============================] - 350s 2s/step - loss: 0.1062 - accuracy: 0.9690 - val_loss: 0.5529 - val_accuracy: 0.8421
Epoch 5/10
212/212 [==============================] - 360s 2s/step - loss: 0.0713 - accuracy: 0.9805 - val_loss: 0.5561 - val_accuracy: 0.8559
Epoch 6/10
212/212 [==============================] - 360s 2s/step - loss: 0.0445 - accuracy: 0.9876 - val_loss: 0.6337 - val_accuracy: 0.8600
Epoch 7/10
212/212 [==============================] - 360s 2s/step - loss: 0.0509 - accuracy: 0.9867 - val_loss: 0.6149 - val_accuracy: 0.8579

In [ ]:
# EVALUATE MODEL

test_loss, test_accuracy = model.evaluate(val_dataset)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

91/91 [==============================] - 33s 364ms/step - loss: 0.6809 - accuracy: 0.8634
Test loss: 0.680923581123352, Test accuracy: 0.8634482622146606


In [17]:
# MODELI KAYDETME

path = 'path-to-save'
# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer')

# Save model
model.save_pretrained(path +'/Model')

In [18]:
# MODELI YUKLEME

# Load tokenizer
#bert_tokenizer = BertTokenizer.from_pretrained(path +'/Tokenizer')

# Load model
#bert_model = TFBertForSequenceClassification.from_pretrained(path +'/Model')

bert_tokenizer = tokenizer
bert_model = model

In [19]:
### DENEME

pred = bert_model.predict(val_dataset)

# pred is of type TFSequenceClassifierOutput
logits = pred.logits

# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)

# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()

label = {
	0: 'Olumsuz',
	1: 'Olumlu',
  2: 'Nötr'
}

# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]

print('Predicted Label :', pred_labels[:10])
print('Actual Label :', Actual[:10])


91/91 [==============================] - 36s 327ms/step
Predicted Label : ['Olumlu', 'Olumsuz', 'Olumlu', 'Olumsuz', 'Olumsuz', 'Olumsuz', 'Nötr', 'Olumsuz', 'Olumlu', 'Olumlu']
Actual Label : ['Olumlu', 'Olumsuz', 'Olumlu', 'Nötr', 'Olumsuz', 'Olumsuz', 'Nötr', 'Olumsuz', 'Nötr', 'Olumlu']


In [20]:
print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support

        Nötr       0.70      0.65      0.68       191
      Olumlu       0.85      0.80      0.82       372
     Olumsuz       0.90      0.94      0.92       887

    accuracy                           0.86      1450
   macro avg       0.82      0.80      0.81      1450
weighted avg       0.86      0.86      0.86      1450



In [21]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

def Get_sentiment(Review, Tokenizer=bert_tokenizer, Model=bert_model):
	# Convert Review to a list if it's not already a list
	if not isinstance(Review, list):
		Review = [Review]

	Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(Review,
																			padding=True,
																			truncation=True,
																			max_length=1024,
																			return_tensors='tf').values()
	prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])

	# Use argmax along the appropriate axis to get the predicted labels
	pred_labels = tf.argmax(prediction.logits, axis = 1)

	# Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
	pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
	return pred_labels


In [22]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

Review = "Vodafone nin sundugu hizmetlerden cok memnum kaldim. Diger operatorlerden daha iyi daha guzel. Vodafone ye gecmek hayatimda aldigim en guzel karardi. Akrabalarima da vodafone ye gecmeleri konusunda ısrarcı olacagim. Fiyatlari cazip ve guzel. Tesekkur ederim."
Get_sentiment(Review)

1/1 [==============================] - 4s 4s/step


['Olumlu']

In [23]:
Review = "kotu berbat."
Get_sentiment(Review)

1/1 [==============================] - 3s 3s/step


['Olumsuz']

In [24]:
Review = "Cok iyi sevdim baya bunu. tesekkurler vodafone"
Get_sentiment(Review)

1/1 [==============================] - 0s 44ms/step


['Olumlu']

In [25]:
Review = "Vodafone yi cok seviyorum. Cok memnun kaldim. Cok tesekkur ederim. İyi ki varsiniz."
Get_sentiment(Review)

1/1 [==============================] - 0s 47ms/step


['Olumlu']

In [26]:
Review = "1 haftadır arama yapamıyorum. Faturalarımı ödememe ragmen sebekeye baglanılamıyor hatasi aliyorum. Bu sorunu lutfen en erken vakitte cozun. Ticari is yapiyorum. Cok zarardayim."
Get_sentiment(Review)

1/1 [==============================] - 0s 46ms/step


['Olumsuz']

In [27]:
Review = "Fiyatlar cok ucuz. Cok guzel cekiyor. Tavsiye ediyorum."
Get_sentiment(Review)

1/1 [==============================] - 0s 43ms/step


['Olumlu']

In [35]:
Review = "ne iyi ne de kotu. Kararsizim"
Get_sentiment(Review)

1/1 [==============================] - 0s 41ms/step


['Olumsuz']